In [1]:
import pandas as pd
from datasets import load_dataset, Audio


df = pd.read_csv("../data/cv-corpus-17.0-delta-2024-03-15/en/validated.tsv", delimiter="\t")
commonvoice = load_dataset("audiofolder", data_dir=f"../data/cv-corpus-17.0-delta-2024-03-15/validated_clips")
commonvoice = commonvoice.cast_column("audio", Audio(sampling_rate=16000)) 



/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1877 examples [00:00, 13157.98 examples/s]


In [ ]:
from transformers import AutoProcessor, Wav2Vec2Model
import torch
import librosa
import soundfile
import pickle
from tqdm import tqdm
import numpy as np
model_checkpoint = "facebook/wav2vec2-base-960h"
model = Wav2Vec2Model.from_pretrained(model_checkpoint)
processor = AutoProcessor.from_pretrained(model_checkpoint)

clip_path = "../data/cv-corpus-17.0-delta-2024-03-15/en"
output_dict = {}
for cv in tqdm(commonvoice["train"]):
    # audio file is decoded on the fly
    inputs = processor(cv["audio"]["array"], sampling_rate=cv["audio"]["sampling_rate"], return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state # i think these are the embeddings that I am going to want
    output_dict[cv['audio']['path'].split("/")[-1].split(".")[0]] = np.mean(last_hidden_states[0].numpy(), axis=0)

with open("commonvoice_embeddings.pkl", 'wb') as file:
    # Use pickle.dump() to serialize and save the dictionary
    pickle.dump(output_dict, file)
    file.close()


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 2/1877 [00:05<1:20:45,  2.58s/it]